# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [2]:
import numpy as np 
import os 
import cv2 
from imageio import imread 
import imageio.v3 as iio
#from skimage.transform  import resize

import warnings
warnings.filterwarnings('ignore')
 
import datetime 
import os
import abc
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
import keras as Keras
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
#train_doc = np.random.permutation(open('../Project_data/train - Copy.csv').readlines())
train_doc = np.random.permutation(open('../Project_data/train - Copy.csv').readlines())
#print(train_doc)
val_doc = np.random.permutation(open('../Project_data/val - Copy.csv').readlines())
#batch_size = 32 #16,32,64 #experiment with the batch size
project_folder='../Project_data/'

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
#creating Model Builder class in order to execute gnerator which has 5 functions - 
#1. Path Settings 
#2. Set Image Properties 
#3. Hyperparameter settings 
#4. Generator functions - Which will return yield 
#5. Batch data which will get called internally in generator data as code is same for remaining sequences as well

class ModelBuilder(metaclass= abc.ABCMeta):
    
    def initialize_path(self,project_folder):
        self.train_doc = np.random.permutation(open(project_folder + '/' + 'train - Copy.csv').readlines())
        self.val_doc = np.random.permutation(open(project_folder + '/' + 'val - Copy.csv').readlines())
        self.train_path = project_folder + '/' + 'train'
        self.val_path =  project_folder + '/' + 'val'
        self.num_train_sequences = len(self.train_doc)
        self.num_val_sequences = len(self.val_doc)
        
    def initialize_image_properties(self,image_height=100,image_width=100):
        self.image_height=image_height
        self.image_width=image_width
        self.channels=3
        self.num_classes=5
        self.total_frames=30
          
    def initialize_hyperparams(self,frames_to_sample=30,batch_size=20,num_epochs=20):
        self.frames_to_sample=frames_to_sample
        self.batch_size=batch_size
        self.num_epochs=num_epochs
        
        
    def generator(self,source_path, folder_list, augment=False):
        img_idx = np.round(np.linspace(0,self.total_frames-1,self.frames_to_sample)).astype(int)
        batch_size=self.batch_size
        while True:
            t = np.random.permutation(folder_list)
            num_batches = len(t)//batch_size
        
            for batch in range(num_batches): 
                batch_data, batch_labels= self.one_batch_data(source_path,t,batch,batch_size,img_idx,augment)
                yield batch_data, batch_labels 

            remaining_seq=len(t)%batch_size
        
            if (remaining_seq != 0):
                batch_data, batch_labels= self.one_batch_data(source_path,t,num_batches,batch_size,img_idx,augment,remaining_seq)
                yield batch_data, batch_labels 
    
    
    def one_batch_data(self,source_path,t,batch,batch_size,img_idx,augment,remaining_seq=0):
    
        seq_len = remaining_seq if remaining_seq else batch_size
    
        batch_data = np.zeros((seq_len,len(img_idx),self.image_height,self.image_width,self.channels)) 
        batch_labels = np.zeros((seq_len,self.num_classes)) 
    
        if (augment): batch_data_aug = np.zeros((seq_len,len(img_idx),self.image_height,self.image_width,self.channels))

        
        for folder in range(seq_len): 
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) 
            for idx,item in enumerate(img_idx): 
                image = iio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                image_resized=cv2.resize(image,(self.image_height,self.image_width  ))
            

                batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255
                batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255
                batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255
            
                if (augment):
                    shifted = cv2.warpAffine(image, 
                                             np.float32([[1, 0, np.random.randint(-30,30)],[0, 1, np.random.randint(-30,30)]]), 
                                            (image.shape[1], image.shape[0]))
                    
                    gray = cv2.cvtColor(shifted,cv2.COLOR_BGR2GRAY)

                    x0, y0 = np.argwhere(gray > 0).min(axis=0)
                    x1, y1 = np.argwhere(gray > 0).max(axis=0) 
                    
                    cropped=shifted[x0:x1,y0:y1,:]
                    
                    image_resized=cv2.resize(cropped,(self.image_height,self.image_width ))

                    batch_data_aug[folder,idx,:,:,0] = (image_resized[:,:,0])/255
                    batch_data_aug[folder,idx,:,:,1] = (image_resized[:,:,1])/255
                    batch_data_aug[folder,idx,:,:,2] = (image_resized[:,:,2])/255
                
            
            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
    
        if (augment):
            batch_data=np.concatenate([batch_data,batch_data_aug])
            batch_labels=np.concatenate([batch_labels,batch_labels])

        
        return(batch_data,batch_labels)
    
    
    def train_model(self, model, augment_data=False):
        train_generator = self.generator(self.train_path, self.train_doc,augment=augment_data)
        val_generator = self.generator(self.val_path, self.val_doc)

        model_name = 'model_init' + '_' + str(datetime.datetime.now()).replace(' ','').replace(':','_') + '/'
    
        if not os.path.exists(model_name):
            os.mkdir(model_name)
        
        filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq="epoch")
        LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)
        callbacks_list = [checkpoint, LR]

        if (self.num_train_sequences%self.batch_size) == 0:
            steps_per_epoch = int(self.num_train_sequences/self.batch_size)
        else:
            steps_per_epoch = (self.num_train_sequences//self.batch_size) + 1

        if (self.num_val_sequences%self.batch_size) == 0:
            validation_steps = int(self.num_val_sequences/self.batch_size)
        else:
            validation_steps = (self.num_val_sequences//self.batch_size) + 1
    
        history=model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=self.num_epochs, verbose=1, 
                            callbacks=callbacks_list, validation_data=val_generator, 
                            validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
        return history

        
    @abc.abstractmethod
    def define_model(self):
        pass

# Start Model Building 

Here we have done several excersize which includes - 

1. Different experiment on Batch size
2. Different experiment on Image size
3. Different Layers & Different modules 
4. Batch Normalization & Dropouts 


## Model - Conv3D  

In [6]:
 class ModelConv3D1(ModelBuilder):
    
    def define_model(self):

        model = Sequential()
        model.add(Conv3D(16, (3, 3, 3), padding='same',
                 input_shape=(self.frames_to_sample,self.image_height,self.image_width,self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(128, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Flatten())
        model.add(Dense(128,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        model.add(Dense(64,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))


        model.add(Dense(self.num_classes,activation='softmax'))

        optimiser = optimizers.Adam()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

In [8]:
conv_3d1=ModelConv3D1()
conv_3d1.initialize_path(project_folder)
conv_3d1.initialize_image_properties(image_height=160,image_width=160)
conv_3d1.initialize_hyperparams(frames_to_sample=30,batch_size=40,num_epochs=1)
conv_3d1_model=conv_3d1.define_model()
conv_3d1_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 30, 160, 160, 16   1312      
                             )                                   
                                                                 
 activation_4 (Activation)   (None, 30, 160, 160, 16   0         
                             )                                   
                                                                 
 batch_normalization_6 (Bat  (None, 30, 160, 160, 16   64        
 chNormalization)            )                                   
                                                                 
 max_pooling3d_4 (MaxPoolin  (None, 15, 80, 80, 16)    0         
 g3D)                                                            
                                                                 
 conv3d_5 (Conv3D)           (None, 15, 80, 80, 32)   

## 1. Model - Conv3D 
#### Image Size - 160 X 160, Batch size - 30, epochs - 5

In [9]:
conv_3d1=ModelConv3D1()
conv_3d1.initialize_path(project_folder)
conv_3d1.initialize_image_properties(image_height=160,image_width=160)
conv_3d1.initialize_hyperparams(frames_to_sample=16,batch_size=30,num_epochs=5)
conv_3d1_model=conv_3d1.define_model()
print("Total Params:", conv_3d1_model.count_params())
conv_3d1.train_model(conv_3d1_model)

Total Params: 1736389
Epoch 1/5


23/23 [==============================] - ETA: 0s - loss: 1.5023 - categorical_accuracy: 0.4344
Epoch 1: saving model to model_init_2024-02-2708_04_16.006109\model-00001-1.50233-0.43439-4.15604-0.21000.h5
23/23 [==============================] - 62s 3s/step - loss: 1.5023 - categorical_accuracy: 0.4344 - val_loss: 4.1560 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 2/5
23/23 [==============================] - ETA: 0s - loss: 0.8509 - categorical_accuracy: 0.6923
Epoch 2: saving model to model_init_2024-02-2708_04_16.006109\model-00002-0.85092-0.69231-6.47878-0.22000.h5
23/23 [==============================] - 46s 2s/step - loss: 0.8509 - categorical_accuracy: 0.6923 - val_loss: 6.4788 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 3/5
23/23 [==============================] - ETA: 0s - loss: 0.5268 - categorical_accuracy: 0.8190
Epoch 3: saving model to model_init_2024-02-2708_04_16.006109\model-00003-0.52683-0.81900-5.54399-0.21000.h5
23

## 2. Model - Conv3D 
#### Image Size - 100 X 100, Batch size - 30, epochs - 5

In [10]:
conv_3d1=ModelConv3D1()
conv_3d1.initialize_path(project_folder)
conv_3d1.initialize_image_properties(image_height=100,image_width=100)
conv_3d1.initialize_hyperparams(frames_to_sample=30,batch_size=30,num_epochs=5)
conv_3d1_model=conv_3d1.define_model()
print("Total Params:", conv_3d1_model.count_params())
history_model  = conv_3d1.train_model(conv_3d1_model)

Total Params: 687813
Epoch 1/5
23/23 [==============================] - ETA: 0s - loss: 1.6273 - categorical_accuracy: 0.4223
Epoch 1: saving model to model_init_2024-02-2708_11_07.834561\model-00001-1.62735-0.42232-3.18315-0.21000.h5
23/23 [==============================] - 174s 8s/step - loss: 1.6273 - categorical_accuracy: 0.4223 - val_loss: 3.1831 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 2/5
23/23 [==============================] - ETA: 0s - loss: 1.0001 - categorical_accuracy: 0.6229
Epoch 2: saving model to model_init_2024-02-2708_11_07.834561\model-00002-1.00005-0.62293-5.57282-0.23000.h5
23/23 [==============================] - 159s 7s/step - loss: 1.0001 - categorical_accuracy: 0.6229 - val_loss: 5.5728 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 3/5
23/23 [==============================] - ETA: 0s - loss: 0.8106 - categorical_accuracy: 0.6772
Epoch 3: saving model to model_init_2024-02-2708_11_07.834561\model-00003-0.81064-0.67722-7.08992-0.17000.h5
23/

## 3. Model - Conv3D 
#### Image Size - 100 X 100, Batch size - 60, epochs - 3

In [11]:
conv_3d1=ModelConv3D1()
conv_3d1.initialize_path(project_folder)
conv_3d1.initialize_image_properties(image_height=100,image_width=100)
conv_3d1.initialize_hyperparams(frames_to_sample=30,batch_size=60,num_epochs=3)
conv_3d1_model=conv_3d1.define_model()
print("Total Params:", conv_3d1_model.count_params())
history_model  = conv_3d1.train_model(conv_3d1_model)

Total Params: 687813
Epoch 1/3
12/12 [==============================] - ETA: 0s - loss: 1.8086 - categorical_accuracy: 0.3529 
Epoch 1: saving model to model_init_2024-02-2708_24_21.744140\model-00001-1.80860-0.35294-1.67041-0.16000.h5
12/12 [==============================] - 184s 16s/step - loss: 1.8086 - categorical_accuracy: 0.3529 - val_loss: 1.6704 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 2/3
12/12 [==============================] - ETA: 0s - loss: 1.1084 - categorical_accuracy: 0.5897 
Epoch 2: saving model to model_init_2024-02-2708_24_21.744140\model-00002-1.10840-0.58974-2.98459-0.16000.h5
12/12 [==============================] - 143s 13s/step - loss: 1.1084 - categorical_accuracy: 0.5897 - val_loss: 2.9846 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 3/3
12/12 [==============================] - ETA: 0s - loss: 0.9321 - categorical_accuracy: 0.6516 
Epoch 3: saving model to model_init_2024-02-2708_24_21.744140\model-00003-0.93214-0.65158-4.59653-0.16000.h

## 4. Model - Conv3D 
#### Image Size - 160 X 160, Batch size - 40, epochs - 5

In [12]:
conv_3d1=ModelConv3D1()
conv_3d1.initialize_path(project_folder)
conv_3d1.initialize_image_properties(image_height=160,image_width=160)
conv_3d1.initialize_hyperparams(frames_to_sample=16,batch_size=40,num_epochs=5)
conv_3d1_model=conv_3d1.define_model()
print("Total Params:", conv_3d1_model.count_params())
conv_3d1.train_model(conv_3d1_model)

Total Params: 1736389
Epoch 1/5
17/17 [==============================] - ETA: 0s - loss: 1.4920 - categorical_accuracy: 0.4540
Epoch 1: saving model to model_init_2024-02-2708_33_01.642530\model-00001-1.49202-0.45400-1.98490-0.16000.h5
17/17 [==============================] - 137s 8s/step - loss: 1.4920 - categorical_accuracy: 0.4540 - val_loss: 1.9849 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 2/5
17/17 [==============================] - ETA: 0s - loss: 0.6531 - categorical_accuracy: 0.7738
Epoch 2: saving model to model_init_2024-02-2708_33_01.642530\model-00002-0.65315-0.77376-3.23808-0.17000.h5
17/17 [==============================] - 120s 7s/step - loss: 0.6531 - categorical_accuracy: 0.7738 - val_loss: 3.2381 - val_categorical_accuracy: 0.1700 - lr: 0.0010
Epoch 3/5
17/17 [==============================] - ETA: 0s - loss: 0.4154 - categorical_accuracy: 0.8416
Epoch 3: saving model to model_init_2024-02-2708_33_01.642530\model-00003-0.41539-0.84163-3.40760-0.13000.h5
17

## 5. Model - Reduced filter size and image size 120 x 120, Augment Data with Dropout
#### Image Size - 120 X 120, Batch size - 30, epochs - 30 

In [13]:
class ModelConv3D3(ModelBuilder):
    
    def define_model(self,filtersize=(3,3,3),dense_neurons=64,dropout=0.25):

        model = Sequential()
        model.add(Conv3D(16, filtersize, padding='same',
                 input_shape=(self.frames_to_sample,self.image_height,self.image_width,self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filtersize, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filtersize, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(128, filtersize, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))


        model.add(Dense(self.num_classes,activation='softmax'))

        optimiser = optimizers.Adam(lr=0.0002)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

In [14]:
conv_3d3=ModelConv3D3()
conv_3d3.initialize_path(project_folder)
conv_3d3.initialize_image_properties(image_height=120,image_width=120)
conv_3d3.initialize_hyperparams(frames_to_sample=16,batch_size=30,num_epochs=30)
conv_3d3_model=conv_3d3.define_model(filtersize=(2,2,2),dense_neurons=256,dropout=0.5)
conv_3d3_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_24 (Conv3D)          (None, 16, 120, 120, 16   400       
                             )                                   
                                                                 
 activation_24 (Activation)  (None, 16, 120, 120, 16   0         
                             )                                   
                                                                 
 batch_normalization_36 (Ba  (None, 16, 120, 120, 16   64        
 tchNormalization)           )                                   
                                                                 
 max_pooling3d_24 (MaxPooli  (None, 8, 60, 60, 16)     0         
 ng3D)                                                           
                                                                 
 conv3d_25 (Conv3D)          (None, 8, 60, 60, 32)    

In [15]:
print("Total Params:", conv_3d3_model.count_params())
history_model3=conv_3d3.train_model(conv_3d3_model,augment_data=True)

Total Params: 1762613
Epoch 1/30
23/23 [==============================] - ETA: 0s - loss: 1.8435 - categorical_accuracy: 0.4178
Epoch 1: saving model to model_init_2024-02-2708_43_40.561252\model-00001-1.84349-0.41780-3.78421-0.16000.h5
23/23 [==============================] - 163s 7s/step - loss: 1.8435 - categorical_accuracy: 0.4178 - val_loss: 3.7842 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 2/30
23/23 [==============================] - ETA: 0s - loss: 1.1711 - categorical_accuracy: 0.5897
Epoch 2: saving model to model_init_2024-02-2708_43_40.561252\model-00002-1.17105-0.58974-6.72155-0.13000.h5
23/23 [==============================] - 142s 6s/step - loss: 1.1711 - categorical_accuracy: 0.5897 - val_loss: 6.7215 - val_categorical_accuracy: 0.1300 - lr: 0.0010
Epoch 3/30
23/23 [==============================] - ETA: 0s - loss: 0.9148 - categorical_accuracy: 0.6644
Epoch 3: saving model to model_init_2024-02-2708_43_40.561252\model-00003-0.91480-0.66440-9.14666-0.14000.h5

23/23 [==============================] - 79s 4s/step - loss: 0.2827 - categorical_accuracy: 0.8959 - val_loss: 2.2265 - val_categorical_accuracy: 0.4100 - lr: 1.6000e-06
Epoch 22/30
23/23 [==============================] - ETA: 0s - loss: 0.3280 - categorical_accuracy: 0.8854
Epoch 22: saving model to model_init_2024-02-2708_43_40.561252\model-00022-0.32798-0.88537-1.72976-0.50000.h5
23/23 [==============================] - 76s 3s/step - loss: 0.3280 - categorical_accuracy: 0.8854 - val_loss: 1.7298 - val_categorical_accuracy: 0.5000 - lr: 1.6000e-06
Epoch 23/30
23/23 [==============================] - ETA: 0s - loss: 0.3226 - categorical_accuracy: 0.8778
Epoch 23: saving model to model_init_2024-02-2708_43_40.561252\model-00023-0.32258-0.87783-1.63166-0.51000.h5
23/23 [==============================] - 70s 3s/step - loss: 0.3226 - categorical_accuracy: 0.8778 - val_loss: 1.6317 - val_categorical_accuracy: 0.5100 - lr: 1.6000e-06
Epoch 24/30
23/23 [==============================] - ETA

## 6. Model - Added few more layers along with dropout 
#### Image Size - 120 X 120, Batch size - 20, epochs - 10 

In [16]:
 class ModelConv3D4(ModelBuilder):
    
    def define_model(self,filtersize=(3,3,3),dense_neurons=64,dropout=0.25):

        model = Sequential()
        model.add(Conv3D(16, filtersize, padding='same',
                 input_shape=(self.frames_to_sample,self.image_height,self.image_width,self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(16, filtersize, padding='same',
                 input_shape=(self.frames_to_sample,self.image_height,self.image_width,self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filtersize, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(32, filtersize, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filtersize, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(64, filtersize, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(128, filtersize, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(128, filtersize, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        

        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))


        model.add(Dense(self.num_classes,activation='softmax'))

        optimiser = optimizers.Adam()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

In [17]:
conv_3d4=ModelConv3D4()
conv_3d4.initialize_path(project_folder)
conv_3d4.initialize_image_properties(image_height=120,image_width=120)
conv_3d4.initialize_hyperparams(frames_to_sample=16,batch_size=20,num_epochs=10)
conv_3d4_model=conv_3d4.define_model(filtersize=(3,3,3),dense_neurons=256,dropout=0.5)
conv_3d4_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_28 (Conv3D)          (None, 16, 120, 120, 16   1312      
                             )                                   
                                                                 
 activation_28 (Activation)  (None, 16, 120, 120, 16   0         
                             )                                   
                                                                 
 batch_normalization_42 (Ba  (None, 16, 120, 120, 16   64        
 tchNormalization)           )                                   
                                                                 
 conv3d_29 (Conv3D)          (None, 16, 120, 120, 16   6928      
                             )                                   
                                                                 
 activation_29 (Activation)  (None, 16, 120, 120, 16  

In [18]:
print("Total Params:", conv_3d4_model.count_params())
history_model4=conv_3d4.train_model(conv_3d4_model,augment_data=True)

Total Params: 2556533
Epoch 1/10
34/34 [==============================] - ETA: 0s - loss: 1.8691 - categorical_accuracy: 0.4020
Epoch 1: saving model to model_init_2024-02-2709_28_50.190618\model-00001-1.86910-0.40196-3.00819-0.22000.h5
34/34 [==============================] - 205s 6s/step - loss: 1.8691 - categorical_accuracy: 0.4020 - val_loss: 3.0082 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 2/10
34/34 [==============================] - ETA: 0s - loss: 1.4222 - categorical_accuracy: 0.5068
Epoch 2: saving model to model_init_2024-02-2709_28_50.190618\model-00002-1.42217-0.50679-3.96332-0.22000.h5
34/34 [==============================] - 191s 6s/step - loss: 1.4222 - categorical_accuracy: 0.5068 - val_loss: 3.9633 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 3/10
34/34 [==============================] - ETA: 0s - loss: 1.3049 - categorical_accuracy: 0.5588
Epoch 3: saving model to model_init_2024-02-2709_28_50.190618\model-00003-1.30490-0.55882-4.33317-0.23000.h5

## 7. Model - Added dropout at convolution layers
#### Image Size - 120 X 120, Batch size - 20, epochs - 20 

In [22]:
class ModelConv3D5(ModelBuilder):
    
    def define_model(self,filtersize=(3,3,3),dense_neurons=64,dropout=0.25):

        model = Sequential()
        model.add(Conv3D(16, filtersize, padding='same',
                 input_shape=(self.frames_to_sample,self.image_height,self.image_width,self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(16, filtersize, padding='same',
                 input_shape=(self.frames_to_sample,self.image_height,self.image_width,self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        model.add(Dropout(dropout))

        model.add(Conv3D(32, filtersize, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(32, filtersize, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        model.add(Dropout(dropout))

        model.add(Conv3D(64, filtersize, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(64, filtersize, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        model.add(Dropout(dropout))

        model.add(Conv3D(128, filtersize, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Conv3D(128, filtersize, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        model.add(Dropout(dropout))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))


        model.add(Dense(self.num_classes,activation='softmax'))

        optimiser = optimizers.Adam()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

In [23]:
conv_3d5=ModelConv3D5()
conv_3d5.initialize_path(project_folder)
conv_3d5.initialize_image_properties(image_height=120,image_width=120)
conv_3d5.initialize_hyperparams(frames_to_sample=16,batch_size=20,num_epochs=20)
conv_3d5_model=conv_3d5.define_model(filtersize=(3,3,3),dense_neurons=256,dropout=0.25)
conv_3d5_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_36 (Conv3D)          (None, 16, 120, 120, 16   1312      
                             )                                   
                                                                 
 activation_36 (Activation)  (None, 16, 120, 120, 16   0         
                             )                                   
                                                                 
 batch_normalization_57 (Ba  (None, 16, 120, 120, 16   64        
 tchNormalization)           )                                   
                                                                 
 conv3d_37 (Conv3D)          (None, 16, 120, 120, 16   6928      
                             )                                   
                                                                 
 activation_37 (Activation)  (None, 16, 120, 120, 16  

In [24]:
print("Total Params:", conv_3d5_model.count_params())
history_model5=conv_3d5.train_model(conv_3d5_model,augment_data=True)

Total Params: 2556533
Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 1.7472 - categorical_accuracy: 0.3967
Epoch 1: saving model to model_init_2024-02-2712_51_11.850275\model-00001-1.74718-0.39668-1.68901-0.24000.h5
34/34 [==============================] - 200s 6s/step - loss: 1.7472 - categorical_accuracy: 0.3967 - val_loss: 1.6890 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 1.3092 - categorical_accuracy: 0.5204
Epoch 2: saving model to model_init_2024-02-2712_51_11.850275\model-00002-1.30920-0.52036-1.94681-0.21000.h5
34/34 [==============================] - 190s 6s/step - loss: 1.3092 - categorical_accuracy: 0.5204 - val_loss: 1.9468 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 1.0475 - categorical_accuracy: 0.6086
Epoch 3: saving model to model_init_2024-02-2712_51_11.850275\model-00003-1.04746-0.60860-2.57951-0.15000.h5

## 8. Model - Reduced the number of parameters  
#### Image Size - 120 X 120, Batch size - 20, epochs - 20

In [25]:
class ModelConv3D7(ModelBuilder):
    
    def define_model(self,dense_neurons=64,dropout=0.25):

        model = Sequential()
        model.add(Conv3D(16, (3, 3, 3), padding='same',
                 input_shape=(self.frames_to_sample,self.image_height,self.image_width,self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, (3, 3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(128, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(self.num_classes,activation='softmax'))

        optimiser = optimizers.Adam(lr=0.0002)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

In [26]:
conv_3d7=ModelConv3D7()
conv_3d7.initialize_path(project_folder)
conv_3d7.initialize_image_properties(image_height=120,image_width=120)
conv_3d7.initialize_hyperparams(frames_to_sample=16,batch_size=20,num_epochs=20)
conv_3d7_model=conv_3d7.define_model(dense_neurons=64,dropout=0.25)
conv_3d7_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_44 (Conv3D)          (None, 16, 120, 120, 16   1312      
                             )                                   
                                                                 
 activation_44 (Activation)  (None, 16, 120, 120, 16   0         
                             )                                   
                                                                 
 batch_normalization_67 (Ba  (None, 16, 120, 120, 16   64        
 tchNormalization)           )                                   
                                                                 
 max_pooling3d_36 (MaxPooli  (None, 8, 60, 60, 16)     0         
 ng3D)                                                           
                                                                 
 conv3d_45 (Conv3D)          (None, 8, 60, 60, 32)   

In [27]:
print("Total Params:", conv_3d7_model.count_params())
history_model7=conv_3d7.train_model(conv_3d7_model,augment_data=True)

Total Params: 504709
Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 1.5341 - categorical_accuracy: 0.4412
Epoch 1: saving model to model_init_2024-02-2714_03_41.008245\model-00001-1.53409-0.44118-2.65844-0.16000.h5
34/34 [==============================] - 169s 5s/step - loss: 1.5341 - categorical_accuracy: 0.4412 - val_loss: 2.6584 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 0.9941 - categorical_accuracy: 0.6267
Epoch 2: saving model to model_init_2024-02-2714_03_41.008245\model-00002-0.99407-0.62670-4.12856-0.16000.h5
34/34 [==============================] - 162s 5s/step - loss: 0.9941 - categorical_accuracy: 0.6267 - val_loss: 4.1286 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 0.7872 - categorical_accuracy: 0.7051
Epoch 3: saving model to model_init_2024-02-2714_03_41.008245\model-00003-0.78718-0.70513-4.02086-0.29000.h5


## 9. Model - CNN + LSTM
#### Image Size - 120 X 120, Batch size - 20, epochs - 20 

In [19]:
from keras.layers import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.layers import LSTM

class RNNCNN1(ModelBuilder):
    
    def define_model(self,lstm_cells=64,dense_neurons=64,dropout=0.25):

        model = Sequential()

        model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                                  input_shape=(self.frames_to_sample,self.image_height,self.image_width,self.channels)))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
        model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
      
        model.add(TimeDistributed(Flatten()))


        model.add(LSTM(lstm_cells))
        model.add(Dropout(dropout))
        
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(Dropout(dropout))
        
        model.add(Dense(self.num_classes, activation='softmax'))
        optimiser = optimizers.Adam()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

In [20]:
rnn_cnn1=RNNCNN1()
rnn_cnn1.initialize_path(project_folder)
rnn_cnn1.initialize_image_properties(image_height=120,image_width=120)
rnn_cnn1.initialize_hyperparams(frames_to_sample=18,batch_size=20,num_epochs=20)
rnn_cnn1_model=rnn_cnn1.define_model(lstm_cells=128,dense_neurons=128,dropout=0.25)
rnn_cnn1_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 18, 120, 120, 16   448       
 ributed)                    )                                   
                                                                 
 time_distributed_1 (TimeDi  (None, 18, 120, 120, 16   64        
 stributed)                  )                                   
                                                                 
 time_distributed_2 (TimeDi  (None, 18, 60, 60, 16)    0         
 stributed)                                                      
                                                                 
 time_distributed_3 (TimeDi  (None, 18, 60, 60, 32)    4640      
 stributed)                                                      
                                                                 
 time_distributed_4 (TimeDi  (None, 18, 60, 60, 32)   

In [21]:
print("Total Params:", rnn_cnn1_model.count_params())
history_model9=rnn_cnn1.train_model(rnn_cnn1_model,augment_data=True)

Total Params: 1657445
Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 1.3508 - categorical_accuracy: 0.4367
Epoch 1: saving model to model_init_2024-02-2710_01_40.642789\model-00001-1.35085-0.43665-1.74414-0.21000.h5
34/34 [==============================] - 120s 3s/step - loss: 1.3508 - categorical_accuracy: 0.4367 - val_loss: 1.7441 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 1.0036 - categorical_accuracy: 0.6169
Epoch 2: saving model to model_init_2024-02-2710_01_40.642789\model-00002-1.00362-0.61689-2.46218-0.22000.h5
34/34 [==============================] - 205s 6s/step - loss: 1.0036 - categorical_accuracy: 0.6169 - val_loss: 2.4622 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 0.8198 - categorical_accuracy: 0.6817
Epoch 3: saving model to model_init_2024-02-2710_01_40.642789\model-00003-0.81983-0.68175-2.32708-0.21000.h5